In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import time
import os
import copy


cudnn.benchmark = True
plt.ion()   # interactive mode

In [3]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
       transforms.RandomResizedCrop(224),
       transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [5]:
 data_dir = '/content/drive/MyDrive/classification_dataset'


image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),data_transforms[x]) for x in ['train']}

dataloaders =  {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=543, shuffle=True, num_workers=2) for x in ['train']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train']}

class_names = image_datasets['train'].classes

print(dataset_sizes)
print(class_names)


image_testsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['test']}
testloaders = {x: torch.utils.data.DataLoader(image_testsets[x], batch_size=543, shuffle=True, num_workers=2) for x in ['test']}

{'train': 543}
['bear', 'butterfly', 'camel', 'chimp', 'duck', 'elephant']


In [6]:
model = torchvision.models.resnet18(pretrained=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [7]:
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor

In [8]:
nodes, _ = get_graph_node_names(model)

In [9]:
print(nodes)

['x', 'conv1', 'bn1', 'relu', 'maxpool', 'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu', 'layer1.0.conv2', 'layer1.0.bn2', 'layer1.0.add', 'layer1.0.relu_1', 'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu', 'layer1.1.conv2', 'layer1.1.bn2', 'layer1.1.add', 'layer1.1.relu_1', 'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.downsample.0', 'layer2.0.downsample.1', 'layer2.0.add', 'layer2.0.relu_1', 'layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.relu', 'layer2.1.conv2', 'layer2.1.bn2', 'layer2.1.add', 'layer2.1.relu_1', 'layer3.0.conv1', 'layer3.0.bn1', 'layer3.0.relu', 'layer3.0.conv2', 'layer3.0.bn2', 'layer3.0.downsample.0', 'layer3.0.downsample.1', 'layer3.0.add', 'layer3.0.relu_1', 'layer3.1.conv1', 'layer3.1.bn1', 'layer3.1.relu', 'layer3.1.conv2', 'layer3.1.bn2', 'layer3.1.add', 'layer3.1.relu_1', 'layer4.0.conv1', 'layer4.0.bn1', 'layer4.0.relu', 'layer4.0.conv2', 'layer4.0.bn2', 'layer4.0.downsample.0', 'layer4.0.downsample.1', 'lay

In [10]:
return_nodes = {
    'flatten' : 'lay'
}

model_FE = create_feature_extractor(model, return_nodes=return_nodes)
model_FE = model_FE.to(device)

In [11]:
model_FE.eval()
for param in model_FE.parameters():
  param.requires_grad = False


for inputs, labels in dataloaders['train']:
  inputs = inputs.to(device)
features = model_FE(inputs)
train_feature_vectors = features['lay'].cpu().numpy()
train_label_vectors = labels.numpy()  
print(train_feature_vectors.shape, train_label_vectors.shape)

(543, 512) (543,)


In [12]:
for inputs, labels in testloaders['test']:
  inputs = inputs.to(device)
features = model_FE(inputs)
test_feature_vectors = features['lay'].cpu().numpy()
test_label_vectors = labels.numpy()  
print(train_feature_vectors.shape, train_label_vectors.shape)

(543, 512) (543,)


In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [14]:
model.eval()
neigh = KNeighborsClassifier(n_neighbors = 10)
neigh.fit(train_feature_vectors, train_label_vectors) 
y_pred = neigh.predict(test_feature_vectors)


# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(test_label_vectors, y_pred)*100)
print(metrics.classification_report(test_label_vectors,y_pred))

Accuracy: 97.5
              precision    recall  f1-score   support

           0       0.95      0.90      0.92        20
           1       1.00      1.00      1.00        20
           2       1.00      1.00      1.00        20
           3       0.95      0.95      0.95        20
           4       0.95      1.00      0.98        20
           5       1.00      1.00      1.00        20

    accuracy                           0.97       120
   macro avg       0.97      0.97      0.97       120
weighted avg       0.97      0.97      0.97       120

